In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

print('Loading Files')
sample_pd =pd.read_csv('result/1300_Pomelotel.csv')
# sample_gpd = gpd.GeoDataFrame(sample_pd, geometry=gpd.GeoSeries.from_wkt(sample_pd['WKT']), crs=4326)
sample_gpd = gpd.GeoDataFrame(sample_pd, geometry=gpd.points_from_xy(sample_pd['longitude'], sample_pd['latitude']), crs=4326)

xmin, ymin, xmax, ymax = sample_gpd.total_bounds

length =  0.0008985 / 20  ### (the distance for 100 metres is 0.0008985, so if you want to get 250 metres, need to multiply 2.5)
wide =  0.0008985 / 20 ### (the distance for 100 metres is 0.0008985, so if you want to get 250 metres, need to multiply 2.5)

print('Processing')
cols = list(np.arange(xmin, xmax + wide, wide))
rows = list(np.arange(ymin, ymax + length, length))

polygons = []
for x in cols[:-1]: ### [:-1] these are used stop -1 of position from the end of list 
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)]))

grid = gpd.GeoDataFrame({'geometry':polygons}, crs=4326)

Loading Files
Processing


In [2]:
grid

,geometry
0,"POLYGON ((106.83547 -6.23811, 106.83551 -6.238..."
1,"POLYGON ((106.83547 -6.23807, 106.83551 -6.238..."
2,"POLYGON ((106.83547 -6.23802, 106.83551 -6.238..."
3,"POLYGON ((106.83547 -6.23798, 106.83551 -6.237..."
4,"POLYGON ((106.83547 -6.23793, 106.83551 -6.237..."
...,...
177,"POLYGON ((106.83601 -6.23771, 106.83605 -6.237..."
178,"POLYGON ((106.83601 -6.23766, 106.83605 -6.237..."
179,"POLYGON ((106.83601 -6.23762, 106.83605 -6.237..."
180,"POLYGON ((106.83601 -6.23757, 106.83605 -6.237..."


In [3]:

grid = grid.sjoin(sample_gpd, how='inner', predicate='intersects')
grid.reset_index(inplace=True)
grid.rename(columns={'index':'id'}, inplace = True)

grid['geometry'] = grid['geometry'].astype(str)
grid.drop(columns=['id','index_right', 'WKT', 'longitude', 'latitude'], errors='ignore', inplace=True)
# grid.rename(columns={'geometry':'WKT'}, inplace=True)

print('Saving to File....')
# grid['partition_column'] = grid.index % 15

# grid.to_parquet('csv_polygon/5x5grid_1300_Pomelotel.parquet', index=False)
grid.to_csv('csv_polygon/5x5grid_1300_Pomelotel.csv', index=False)
#ddf = da.from_pandas(grid, chunksize=3000000)
#ddf.to_parquet('/home/nivag/grid_folder/grid_40x40/')

Saving to File....


/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
